# Integration Examples using QMCPy package

In [1]:
from qmcpy import *

from numpy import arange

## Keister Example
Keister Integrand:
- $y_i = \pi^{d/2} * \cos(||x_i||_2)$
    
Gaussian True Measure:
- $\mathcal{N}(0,\frac{1}{2})$
    
Sobol Discrete Distribution:
- $x_j \overset{lds}{\sim}    \mathcal{U}(0,1)$

In [2]:
dim = 3
integrand = Keister(dim)
discrete_distrib = Sobol(rng_seed=7)
true_measure = Gaussian(dim, variance=1 / 2)
stopping_criterion = CLTRep(discrete_distrib, true_measure, abs_tol=.05)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 2.1477         
Keister (Integrand Object)
Sobol (Discrete Distribution Object)
	mimics          StdUniform
Gaussian (True Measure Object)
	dimension       3
	mu              0
	sigma           0.707
CLTRep (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           1073741824
	inflate         1.200
	alpha           0.010
MeanVarDataRep (AccumData Object)
	n               32
	n_total         32.0
	confid_int      [ 2.126  2.169]
	time_total      0.010
	r               16



## Asian Option Pricing Example
### Single Level 
Asian Call Option Integrand
- $S_i(t_j)=S(0)e^{(r-\frac{\sigma^2}{2})t_j+\sigma\mathcal{B}(t_j)}$
- discounted put payoff $= max(K-\frac{1}{d}\sum_{j=0}^{d-1} S(jT/d))\;,\: 0)$

Brownian Motion True Measure: 
- $\:\: \mathcal{B}(t_j)=B(t_{j-1})+Z_j\sqrt{t_j-t_{j-1}} \;$ for $\;Z_j \sim \mathcal{N}(0,1)$

Lattice Discrete Distribution:
- $\:\: x_j  \overset{lds}{\sim}    \mathcal{U}(0,1)$

In [3]:
time_vec = [arange(1 / 64, 65 / 64, 1 / 64)]
dim = [len(tv) for tv in time_vec]

discrete_distrib = Lattice(rng_seed=7)
true_measure = BrownianMotion(dim, time_vector=time_vec)
integrand = AsianCall(true_measure,
                volatility = .5,
                start_price = 30,
                strike_price = 25,
                interest_rate = .01,
                mean_type = 'geometric')
stopping_criterion = CLTRep(discrete_distrib, true_measure, abs_tol=.05)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 5.8356         
AsianCall (Integrand Object)
	volatility      0.500
	start_price     30
	strike_price    25
	interest_rate   0.010
	mean_type       geometric
	exercise_time   1.0
Lattice (Discrete Distribution Object)
	mimics          StdUniform
BrownianMotion (True Measure Object)
	dimension       64
	time_vector     [ 0.016  0.031  0.047 ...  0.969  0.984  1.000]
CLTRep (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           1073741824
	inflate         1.200
	alpha           0.010
MeanVarDataRep (AccumData Object)
	n               2048
	n_total         2048.0
	confid_int      [ 5.833  5.838]
	time_total      0.434
	r               16



## Asian Option Pricing Example
### Multi-Level
$Y_0 = 0$

$Y_1$ = Asian Option Monitored at $t=[\frac{1}{4}, \frac{1}{2}, \frac{3}{4}, 1]$

$Y_2$ = Asian Option Monitored at $t=[\frac{1}{16}, \frac{1}{8}, ... , 1]$

$Y_3$ = Asian Option Monitored at $t=[\frac{1}{64}, \frac{1}{32}, ... , 1]$

$Z_1 = \mathbb{E}[Y_1-Y_0] + \mathbb{E}[Y_2-Y_1] + \mathbb{E}[Y_3-Y_2] = \mathbb{E}[Y_3]$

In [4]:
time_vec = [arange(1 / 4, 5 / 4, 1 / 4),
                 arange(1 / 16, 17 / 16, 1 / 16),
                 arange(1 / 64, 65 / 64, 1 / 64)]
dim = [len(tv) for tv in time_vec]

discrete_distrib = IIDStdGaussian(rng_seed=7)
true_measure = BrownianMotion(dim, time_vector=time_vec)
integrand = AsianCall(true_measure,
                volatility = .5,
                start_price = 30,
                strike_price = 25,
                interest_rate = .01,
                mean_type = 'geometric')
stopping_criterion = CLT(discrete_distrib, true_measure, abs_tol=.05, n_max = 1e10)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 5.8276         
AsianCall (Integrand Object)
	volatility      [ 0.500  0.500  0.500]
	start_price     [30 30 30]
	strike_price    [25 25 25]
	interest_rate   [ 0.010  0.010  0.010]
	mean_type       ['geometric' 'geometric' 'geometric']
	exercise_time   [ 1.000  1.000  1.000]
IIDStdGaussian (Discrete Distribution Object)
	mimics          StdGaussian
BrownianMotion (True Measure Object)
	dimension       [ 4 16 64]
	time_vector     [array([ 0.250,  0.500,  0.750,  1.000])
	                array([ 0.062,  0.125,  0.188, ...,  0.875,  0.938,  1.000])
	                array([ 0.016,  0.031,  0.047, ...,  0.969,  0.984,  1.000])]
CLT (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           10000000000
	inflate         1.200
	alpha           0.010
MeanVarData (AccumData Object)
	n               [ 246550.000  34323.000  5430.000]
	n_total         289375.0
	confid_int      [ 5.779  5.876]
	time_total      0.185

